In [1]:
import joblib
from FV2 import AllSystem
from configV2 import config
demo = joblib.load('data/demo.pkl')
f = joblib.load('model/allsystem.pkl')

In [2]:
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']

In [3]:
icg_input

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
139-017,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,32.0,0.0911,20.854397,74.689774,80.0,920.0,2.75,2.5,36.0


In [4]:
icg_input[['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']]

,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr
139-017,32.0


# 利用兩次試算的差異(當作調幅)達成推薦效果

In [5]:
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f.inference(icg_input.copy(),c620_feed.copy(),t651_feed.copy())

icg_input2 = icg_input.copy()
icg_input2['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = 980

#C620 BZ in SideDraw目標是越低越好(min 70%)，但若要消耗T651原料時(T651流量上升)，須將C620的測取BZ濃度目標值提升。
#=> 影響C620 BZ in SideDraw 變數為 T651 flow。昨日副廠有提到T651流量大於150m3/hr時，C620測取苯濃度需操作在80 ~ 85%。
#=>想法為 推薦功能中BZ in SideDraw 設定值為{if (T651_m3/hr >150[A]), BZ in SideDraw = 80[B]; (T651_m3/hr <150), BZ in SideDraw = 70 }，其中A 與B的確切數值請弘暉提供。
if icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr'].values[0] > 150:
    icg_input2['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = 85
    print('設定Benzene in C620 Sidedraw_wt%=85')
else:
    icg_input2['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = 70
    print('設定Benzene in C620 Sidedraw_wt%=70')
icg_input2['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10

c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f.inference(icg_input2.copy(),c620_feed.copy(),t651_feed.copy())

設定Benzene in C620 Sidedraw_wt%=70


# 推薦的c620溫度調幅

In [6]:
(c620_op2-c620_op1).iloc[:,-2:]

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
139-017,0.684555,0.471985
